In [246]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

In [247]:
#NorthWind database
# Define your connection string
conn_str_nw = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=KAM\SQLEXPRESS;'
    r'DATABASE=NorthWind;'
    r'Trusted_Connection=yes;'
)

# URL-encode the connection string
quoted_conn_str_nw = urllib.parse.quote_plus(conn_str_nw)

# Create the SQLAlchemy engine using the URL-encoded connection string
engine_nw = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str_nw}")

# Read the SQL query into a DataFrame using the engine
customerNW = pd.read_sql_query("SELECT * FROM Customers", engine_nw)

#dispose of the engine
engine_nw.dispose()

#northwind database drop column en rename
df_customerNW = customerNW.drop(['ContactTitle'], axis=1) 
df_customerNW.rename(columns={
    'CustomerID': 'Id',
    'ContactName': 'FullName',
    'PostalCode':'Postalcode',
    'Phone':'PhoneNumber',
    'country':'Country'
}, inplace=True)
#adding column with value 'NW'
df_customerNW['Database'] = "NW"

In [248]:
#adventure_works database
conn_str_aw = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=KAM\SQLEXPRESS;'
    r'DATABASE=AdventureWorks2019;'
    r'Trusted_Connection=yes;'
)

# URL-encode the connection string
quoted_conn_str_aw = urllib.parse.quote_plus(conn_str_aw)

# Create the SQLAlchemy engine using the URL-encoded connection string
engine_aw = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str_aw}")

#Tabellen pakken
customerAW = pd.read_sql_query("SELECT * FROM Sales.Customer", engine_aw)
companyAW = pd.read_sql_query("SELECT * FROM Sales.Store", engine_aw)
personAW = pd.read_sql_query("SELECT * FROM Person.Person", engine_aw)
businessEntityAddress = pd.read_sql_query("SELECT * FROM Person.BusinessEntityAddress", engine_aw)
address = pd.read_sql_query("SELECT AddressID, AddressLine1, City, StateProvinceID, PostalCode FROM Person.Address", engine_aw)
stateProvencie= pd.read_sql_query("SELECT * FROM Person.StateProvince", engine_aw)
phonenumberAW = pd.read_sql_query("SELECT * FROM Person.PersonPhone", engine_aw)
country = pd.read_sql_query("SELECT * FROM Person.CountryRegion", engine_aw)

#Tabellen mergen
df_customer_company = pd.merge(customerAW, companyAW, how='left', left_on='StoreID', right_on='BusinessEntityID')
dfCustomer_PCC= pd.merge(df_customer_company,personAW, how='left', left_on='PersonID', right_on='BusinessEntityID')

dfCustomer_PCC['FullName'] = dfCustomer_PCC['FirstName'] + ' ' +dfCustomer_PCC['MiddleName']+ ' ' + dfCustomer_PCC['LastName']
dfCustomer_PCC=dfCustomer_PCC.loc[:, ['CustomerID','TerritoryID', 'Name','FullName','BusinessEntityID_y']]

dfCustomerPCCA=pd.merge(dfCustomer_PCC,businessEntityAddress, how='left', left_on='BusinessEntityID_y', right_on='BusinessEntityID')
dfCustomerPCCAA=pd.merge(dfCustomerPCCA,address, how='left', left_on='AddressID', right_on='AddressID')
dfCustomerPCCAA.rename(columns={
    'CustomerID': 'Id',
    'Name':'CompanyName',
    'AddressLine1':'Address',
    'PostalCode':'Postalcode'
}, inplace=True)

dfCustomerPCCAAS=pd.merge(dfCustomerPCCAA,stateProvencie, how='left', left_on='StateProvinceID', right_on='StateProvinceID')
dfCustomerPCCAAS.rename(columns={
    'Name': 'Region',
}, inplace=True)

dfCustomerPCCAAS = dfCustomerPCCAAS.loc[:, ['Id', 'CompanyName', 'FullName','Address','City','Region','Postalcode','CountryRegionCode','BusinessEntityID']]

dfCustomerPCCAASC=pd.merge(dfCustomerPCCAAS,country, how='left', left_on='CountryRegionCode', right_on='CountryRegionCode')
dfCustomerPCCAASC.rename(columns={
    'Name': 'Country',
}, inplace=True)
dfCustomerAW=pd.merge(dfCustomerPCCAASC,phonenumberAW, how='left', left_on='BusinessEntityID', right_on='BusinessEntityID')

#enine dispose
engine_aw.dispose()

#adding column with value 'AW'
dfCustomerAW['Database'] = "AW"

# Hier selecteer ik allen de column die ik nodig heb.
df_customerAW = dfCustomerAW.loc[:, ['Id','Database', 'CompanyName', 'FullName','Address','City','Region','Country','Postalcode','PhoneNumber']]


In [249]:
#aenc database
engine = create_engine('access+pyodbc:///?odbc_connect=' +
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    'DBQ=C:\\Users\\ozank\\Downloads\\aenc.accdb;')
#datapakken
customerAE = pd.read_sql_query("select * from customer", engine)
states = pd.read_sql_query("select * from state", engine)

#merge customer en states
df_customerAE = pd.merge(customerAE, states, how='left', left_on='state', right_on='state_id')

#namen samenvoegen
df_customerAE['FullName'] = df_customerAE['fname'] + ' ' + df_customerAE['lname']
df_customerAE = df_customerAE.drop(['fname', 'lname', 'state_id', 'region', 'state_capital', 'state'], axis=1)

#column namen veranderen
df_customerAE.rename(columns={
    'id': 'Id',
    'company_name': 'CompanyName',
    'address': 'Address',
    'city': 'City',
    'state_name': 'Region',
    'country': 'Country',
    'zip': 'Postalcode',
    'phone': 'PhoneNumber'
}, inplace=True)

df_customerAE['Database'] = "AE"

In [250]:
#alle tabelen samenvoegen
customer = pd.concat([df_customerAE, df_customerNW, df_customerAW])

In [251]:
customer

,Id,Address,City,Postalcode,PhoneNumber,CompanyName,Region,Country,FullName,Database,Fax
0,101,3114 Pioneer Avenue,Rutherford,07070,2015558966,The Power Group,New Jersey,USA,Michaels Devlin,AE,NaN
1,102,1033 Whippany Road,New York,10154,2125558725,AMF Corp.,New York,USA,Beth Reiser,AE,NaN
2,103,1990 Windsor Street,Paoli,19301,2155556513,Darling Associates,Pennsylvania,USA,Erin Niedringhaus,AE,NaN
3,104,550 Dundas Street East,Knoxville,37919,6155555463,P.S.C.,Tennessee,USA,Meghan Mason,AE,NaN
4,105,1210 Highway 36,Carmel,46032,3175558437,Amo & Sons,Indiana,USA,Laura McCarthy,AE,NaN
...,...,...,...,...,...,...,...,...,...,...,...
19839,30114,NaN,NaN,NaN,NaN,Recreation Toy Store,NaN,NaN,NaN,AW,NaN
19840,30115,NaN,NaN,NaN,NaN,Retreat Inn,NaN,NaN,Dora P. Verdad,AW,NaN
19841,30116,NaN,NaN,NaN,NaN,Technical Parts Manufacturing,NaN,NaN,Wanda F. Vernon,AW,NaN
19842,30117,NaN,NaN,NaN,NaN,Totes & Baskets Company,NaN,NaN,Robert R. Vessa,AW,NaN
